In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate

In [2]:
!head -n 2 google*
!wc -l google*

==> googleplaystore.csv <==
App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up

==> googleplaystore_user_reviews.csv <==
App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
10 Best Foods for You,"I like eat delicious food. That's I'm cooking food myself, case ""10 Best Foods"" helps lot, also ""Best Before (Shelf Life)""",Positive,1.0,0.5333333333333333
  10842 googleplaystore.csv
  64296 googleplaystore_user_reviews.csv
  75138 total


In [3]:
gps = pd.read_csv("googleplaystore.csv", index_col = 0)
gps_ur = pd.read_csv("googleplaystore_user_reviews.csv", index_col = 0)

In [4]:
pd.unique(gps["Size"].str.replace(r'[0-9]+', '')) # find units of measure and notation types

array(['M', '.M', 'Varies with device', 'k', '.k'], dtype=object)

In [5]:
## SOLVED
# gps[gps['Size'].str.contains("\+")] # Una sola riga
# A questa riga manca originariamente la seconda colonna (Category)
# di conseguenza le colonne risultano shiftate
# Modifichiamo manualmente il csv aggiungendo una virgola in piu'

In [6]:
gps[gps['Size'].str.contains("Varies with device")].shape

(1695, 12)

In [7]:
gps.loc[gps['Size'].str.contains("Varies with device"), "Size"] = '-1'

In [8]:
sz = pd.DataFrame(gps.Size.str.split('.').tolist())
print(sz.shape) # no more than two columns --> no more than two dots
sz[1].unique() # always just one number after the dot -> no thousands

(10841, 2)


array([None, '7M', '8M', '6M', '1M', '5M', '2M', '0M', '4M', '9M', '3M',
       '5k'], dtype=object)

In [9]:
# Only one app has decimal size with kilobytes
gps[gps['Size'].str.contains("\.[0-9]k")] # the only one

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Essential Resources,LIBRARIES_AND_DEMO,4.6,237,8.5k,"50,000+",Free,0,Everyone,Libraries & Demo,"September 27, 2017",1.02,7.1 and up


In [10]:
gps["size_n"] = pd.to_numeric(gps.Size.str.extract('(^\d+(\.\d)?)')[0])
# Non perdiamo dati, quelli con size non determinabile
# sono sempre quelli (1695)
# Tutte le misure che non sono consone sono qui gia' trasformate in null,
# quindi anche il punto (3) "Varies with size" --> NaN 
# Poiche' era stato trasformato in "-1", che non viene matchato dalla regex
sum(pd.isna(gps["size_n"]))

1695

In [11]:
# def flag_size(series):
#     if (series.str.contains("k")):
#         return 1024
#     elif (series.str.contains("M")):
#         return 1024 ** 2
#     else:
#         return 0
#gps['size_mag'] = gps.apply(flag_df_size, axis = 1)
#gps['size_mag'] = gps['Size'].apply(flag_size, axis = 1)

# Single line code instead of function
gps['size_mag'] = pd.np.where(gps.Size.str.contains("k"), 1024,
                              pd.np.where(gps.Size.str.contains("M"), 1024 ** 2,
                                          0))

#### (1) Convert app sizes to a number and
#### (3) Transform “Varies with device” in `Size` column into a missing value

In [12]:
gps["Size"] = gps.size_n * gps.size_mag
df1 = gps.drop(gps.columns[[-1, -2]], axis = 1, inplace = False) # drop 'size_mag', 'size_n'
df1.tail(10)

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
payermonstationnement.fr,MAPS_AND_NAVIGATION,NaN,38,10276044.8,"5,000+",Free,0,Everyone,Maps & Navigation,"June 13, 2018",2.0.148.0,4.0 and up
FR Tides,WEATHER,3.8,1195,595968.0,"100,000+",Free,0,Everyone,Weather,"February 16, 2014",6.0,2.1 and up
Chemin (fr),BOOKS_AND_REFERENCE,4.8,44,633856.0,"1,000+",Free,0,Everyone,Books & Reference,"March 23, 2014",0.8,2.2 and up
FR Calculator,FAMILY,4.0,7,2726297.6,500+,Free,0,Everyone,Education,"June 18, 2017",1.0.0,4.1 and up
FR Forms,BUSINESS,NaN,0,10066329.6,10+,Free,0,Everyone,Business,"September 29, 2016",1.1.5,4.0 and up
Sya9a Maroc - FR,FAMILY,4.5,38,55574528.0,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3774873.6,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
Parkinson Exercices FR,MEDICAL,NaN,3,9961472.0,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [13]:
pd.unique(df1.Installs)
#df1[df1.Installs.str.contains("Free")] # Una sola riga | solved, dovuto alla mancata virgola nel csv.

array(['10,000+', '500,000+', '5,000,000+', '50,000,000+', '100,000+',
       '50,000+', '1,000,000+', '10,000,000+', '5,000+', '100,000,000+',
       '1,000,000,000+', '1,000+', '500,000,000+', '50+', '100+', '500+',
       '10+', '1+', '5+', '0+', '0'], dtype=object)

#### (2) Convert the number of installs to a number

In [14]:
df1.Installs = df1.Installs.str.replace(r'[\,\+]', '').astype(int)
df1.sample(20)

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
CE BF,LIFESTYLE,NaN,0,25165824.0,50,Free,0,Everyone,Lifestyle,"January 2, 2018",1.0.1,4.1 and up
Dresses Ideas & Fashions +3000,BEAUTY,4.5,473,8598323.2,100000,Free,0,Mature 17+,Beauty,"March 1, 2017",1.0.2.0,1.6 and up
R Programing Offline Tutorial,BOOKS_AND_REFERENCE,5.0,4,4089446.4,1000,Free,0,Everyone,Books & Reference,"March 15, 2018",1.0.0,4.1 and up
Halalguur,DATING,4.3,6,9437184.0,100,Free,0,Mature 17+,Dating,"July 26, 2018",3.7,4.0.3 and up
Random Video Chat App With Strangers,DATING,NaN,3,5033164.8,1000,Free,0,Mature 17+,Dating,"July 17, 2018",1.,4.0 and up
DP Maker,PHOTOGRAPHY,4.2,784,18874368.0,100000,Free,0,Everyone,Photography,"April 29, 2018",1.00.15,4.1 and up
Wide address pocket prep,MEDICAL,4.5,513,12582912.0,5000,Free,0,Everyone,Medical,"July 11, 2018",4.5.2,4.4 and up
USMLE Step 2 CK Flashcards,FAMILY,5.0,1,41943040.0,10,Paid,$19.99,Everyone,Education,"May 26, 2014",2.47,2.2 and up
Sworkit: Workouts & Fitness Plans,HEALTH_AND_FITNESS,4.6,109756,81788928.0,5000000,Free,0,Everyone,Health & Fitness,"August 1, 2018",8.2.0,4.0.3 and up


#### (3) Transform “Varies with device” into a missing value

In [15]:
df1.replace({'Varies with device' : None}, inplace = True)

In [16]:
# Extract only the version number (that is a sequence of numbers and dots)
df1["Ver"] = df1["Current Ver"].str.extract('(\d+(\.\d+)*)')[0]
df1["Ver_compatib"] = df1["Android Ver"].str.extract('(\d+(\.\d+)*)')[0]

In [17]:
df1.head(5)

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Ver,Ver_compatib
App,,,,,,,,,,,,,,
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19922944.0,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,1.0.0,4.0.3
Coloring book moana,ART_AND_DESIGN,3.9,967,14680064.0,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,2.0.0,4.0.3
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,9122611.2,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,1.2.4,4.0.3
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,26214400.0,50000000,Free,0,Teen,Art & Design,"June 8, 2018",None,4.2 and up,NaN,4.2
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2936012.8,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,1.1,4.4


#### (4) Convert Current Ver and Android Ver into a dotted number

In [18]:
df1["Current Ver"] = df1.Ver
df1["Android Ver"] = df1.Ver_compatib
df2 = df1.drop(df1.columns[[-1, -2]], axis = 1, inplace = False)

In [19]:
# Check operation's behaviour (on corner cases), select few rows to inspect 
idx10 = df1["Android Ver"].str.count(r"\.").nlargest(keep = "first").index
idx11 = df1["Android Ver"].str.count(r"\.").nsmallest(keep = "last").index
idx1  = idx10.union(idx11)
idx20 = df1["Current Ver"].str.count(r"\.").nlargest(keep = "last").index
idx21 = df1["Current Ver"].str.count(r"\.").nsmallest(keep = "first").index
idx2  = idx20.union(idx21)

# Combine all indexes
idx   = idx1.union(idx2)

In [20]:
pd.concat([df1.loc[idx, ["Current Ver", "Android Ver"]], df2.loc[idx, ["Current Ver", "Android Ver"]]], axis = 1)

,Current Ver,Android Ver,Current Ver,Android Ver
App,,,,
"Calculator - free calculator, multi calculator app",8.0.1.8.0629.1,4.2,8.0.1.8.0629.1,4.2
Coloring book moana,2.0.0,4.0.3,2.0.0,4.0.3
Coloring book moana,2.0.0,4.0.3,2.0.0,4.0.3
ExDialer - Dialer & Contacts,196,2.1,196,2.1
FR Calculator,1.0.0,4.1,1.0.0,4.1
FR Forms,1.1.5,4.0,1.1.5,4.0
File Manager -- Take Command of Your Files Easily,7.0.7.1.0625.1,5.0,7.0.7.1.0625.1,5.0
Fr. Mike Schmitz Audio Teachings,1.0,4.1,1.0,4.1
How To Color Disney Princess - Coloring Pages,1,4.0,1,4.0


In [21]:
df2[df2.index.duplicated(keep = False)].sort_index().shape # total duplicates
df2[df2.index.duplicated(keep = "last")].sort_index().shape # unique duplicates

(1181, 12)

#### (5) Remove duplicates

In [22]:
# Select random duplicated apps to double check if results are correct
idx_dup = df2[df2.index.duplicated()].sample(n = 5, random_state = 321).index
df2.loc[idx_dup]

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
EMT PASS,MEDICAL,3.4,51,2516582.4,1000,Paid,$29.99,Everyone,Medical,"October 22, 2014",2.0.2,4.0
EMT PASS,MEDICAL,3.4,51,2516582.4,1000,Paid,$29.99,Everyone,Medical,"October 22, 2014",2.0.2,4.0
Lose It! - Calorie Counter,HEALTH_AND_FITNESS,4.4,69395,NaN,10000000,Free,0,Everyone,Health & Fitness,"August 3, 2018",NaN,NaN
Lose It! - Calorie Counter,HEALTH_AND_FITNESS,4.4,69395,NaN,10000000,Free,0,Everyone,Health & Fitness,"August 3, 2018",NaN,NaN
Lose It! - Calorie Counter,HEALTH_AND_FITNESS,4.4,69395,NaN,10000000,Free,0,Everyone,Health & Fitness,"August 3, 2018",NaN,NaN
Block Craft 3D: Building Simulator Games For Free,GAME,4.5,946926,59768832.0,50000000,Free,0,Everyone,Simulation,"March 5, 2018",2.10.2,4.0.3
Block Craft 3D: Building Simulator Games For Free,GAME,4.5,946926,59768832.0,50000000,Free,0,Everyone,Simulation,"March 5, 2018",2.10.2,4.0.3
Block Craft 3D: Building Simulator Games For Free,FAMILY,4.5,944661,59768832.0,50000000,Free,0,Everyone,Simulation,"March 5, 2018",2.10.2,4.0.3
Temple Run 2,GAME,4.3,8118609,65011712.0,500000000,Free,0,Everyone,Action,"July 5, 2018",1.49.1,4.0


In [23]:
# Remove duplicates, keep last (most recent ?)
df3 = df2[~df2.index.duplicated(keep = 'last')] # 10841 - 1181 = 9660
df3.loc[idx_dup]

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
EMT PASS,MEDICAL,3.4,51,2516582.4,1000,Paid,$29.99,Everyone,Medical,"October 22, 2014",2.0.2,4.0
Lose It! - Calorie Counter,HEALTH_AND_FITNESS,4.4,69395,NaN,10000000,Free,0,Everyone,Health & Fitness,"August 3, 2018",NaN,NaN
Block Craft 3D: Building Simulator Games For Free,FAMILY,4.5,944661,59768832.0,50000000,Free,0,Everyone,Simulation,"March 5, 2018",2.10.2,4.0.3
Temple Run 2,GAME,4.3,8116142,65011712.0,500000000,Free,0,Everyone,Action,"July 5, 2018",1.49.1,4.0
Cartoon Network App,FAMILY,4.0,119173,NaN,10000000,Free,0,Everyone 10+,Video Players & Editors;Music & Video,"June 21, 2018",NaN,NaN


In [24]:
print(df3.shape)
cat = df3.groupby('Category').size()
sum(cat) # One category is missing 
df3.loc[df3['Category'].isnull()] # sempre quella di prima !

(9660, 12)


,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Life Made WI-Fi Touchscreen Photo Frame,NaN,1.9,19,3145728.0,1000,Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0


#### (6)-(7) Number of apps and average ratings for each category

In [25]:
agg1 = df3.groupby("Category").aggregate({"Rating": 'mean', "Category": 'count'}).rename(columns = {"Rating": "Rating_avg", "Category": "count"})
print(agg1["count"].sum()) # one missing (App: "Life Made WiFi...")
agg1

9659


,Rating_avg,count
Category,,
ART_AND_DESIGN,4.359322,61
AUTO_AND_VEHICLES,4.190411,85
BEAUTY,4.278571,53
BOOKS_AND_REFERENCE,4.344970,222
BUSINESS,4.096565,419
COMICS,4.181481,56
COMMUNICATION,4.121401,316
DATING,3.980451,170
EDUCATION,4.351485,102


In [26]:
df3[df3.index.str.contains("Pixel Draw")]

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2936012.8,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4


#### (8) Create two dataframes: one for the genres (?) and one bridging

In [27]:
genres_df = pd.DataFrame({"Genre": df3.Genres.str.split(";", expand = True).stack().unique()})
# genres_df = pd.DataFrame({"Genre": pd.unique(df3.Genres)})
genres_df

,Genre
0,Art & Design
1,Creativity
2,Auto & Vehicles
3,Beauty
4,Books & Reference
5,Business
6,Comics
7,Communication
8,Dating
9,Education


In [28]:
bridging_df = df3.Genres.str.split(";").apply(pd.Series).reset_index() \
    .melt(id_vars = "App", var_name = "foo", value_name = "Genre") \
    .drop("foo", axis = 1) \
    .dropna()
bridging_df.head(5)

,App,Genre
0,Photo Editor & Candy Camera & Grid & ScrapBook,Art & Design
1,"U Launcher Lite – FREE Live Cool Themes, Hide ...",Art & Design
2,Sketch - Draw & Paint,Art & Design
3,Pixel Draw - Number Art Coloring Book,Art & Design
4,Paper flowers instructions,Art & Design


In [29]:
# Check results: first and last 5 rows
bridging_df.groupby("App").count().sort_values(by = ["Genre", "App"], ascending = [False, True])\
.iloc[np.r_[0:5, -5:0]]
#.sample(10, random_state = 1337)

,Genre
App,
ABC Kids - Tracing & Phonics,2
ABC Preschool Free,2
ABCmouse.com,2
AJ Academy: Amazing Animals,2
Adult Glitter Color by Number Book - Sandbox Pages,2
"뽕티비 - 개인방송, 인터넷방송, BJ방송",1
💎 I'm rich,1
"💘 WhatsLov: Smileys of love, stickers and GIF",1
📏 Smart Ruler ↔️ cm/inch measuring for homework!,1


In [30]:
genres_dummy = df3.iloc[:, df3.columns.get_loc('Genres')].str.get_dummies(sep=';').astype("bool")
genres_dummy.add_prefix("genre_")
genres_dummy.head(5)

,Action,Action & Adventure,Adventure,Arcade,Art & Design,Auto & Vehicles,Beauty,Board,Books & Reference,Brain Games,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
App,,,,,,,,,,,,,,,,,,,,,
Photo Editor & Candy Camera & Grid & ScrapBook,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Sketch - Draw & Paint,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Pixel Draw - Number Art Coloring Book,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Paper flowers instructions,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [31]:
print("No_duplicates' df shape: {}\nDummy_genres' df shape: {}".format(df3.shape, genres_dummy.shape))
genres_dummy.sum(axis = 1).sort_values(ascending = False).iloc[np.r_[0:5, -5:0]] # missing genre in "Life Made WI-Fi*"

No_duplicates' df shape: (9660, 12)
Dummy_genres' df shape: (9660, 53)


App
Unicorn Pokez - Color By Number                     2
Hactar Go                                           2
Steam: Rails to Riches                              2
No.Diamond – Colors by Number                       2
SweetLand — Family Board Game                       2
Ruler cm                                            1
📏 Smart Ruler ↔️ cm/inch measuring for homework!    1
CM Secure Browser (Authorized)                      1
Photo Editor & Candy Camera & Grid & ScrapBook      1
Life Made WI-Fi Touchscreen Photo Frame             0
dtype: int64

#### (9) Create dummy columns in the original dataframe for column "Genres"

In [32]:
df4 = df3.join(genres_dummy)
df4.head(5)

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
App,,,,,,,,,,,,,,,,,,,,,
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19922944.0,10000,Free,0,Everyone,Art & Design,"January 7, 2018",...,False,False,False,False,False,False,False,False,False,False
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,9122611.2,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",...,False,False,False,False,False,False,False,False,False,False
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,26214400.0,50000000,Free,0,Teen,Art & Design,"June 8, 2018",...,False,False,False,False,False,False,False,False,False,False
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2936012.8,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",...,False,False,False,False,False,False,False,False,False,False
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5872025.6,50000,Free,0,Everyone,Art & Design,"March 26, 2017",...,False,False,False,False,False,False,False,False,False,False


#### (10) For each genre, compute the average rating. What is the highest rated genre?

In [33]:
df5 = bridging_df.merge(df3['Rating'].reset_index(), on = "App")
df5.dropna().groupby("Genre").mean().sort_values(by = "Rating", ascending = False) # Events with avg rating of 4.436

,Rating
Genre,
Events,4.435556
Puzzle,4.370732
Brain Games,4.358730
Art & Design,4.350000
Books & Reference,4.343275
Personalization,4.332215
Word,4.318182
Creativity,4.312903
Parenting,4.300000


#### (11) For each app, compute the approximate income, obtain as a product of number of installs and price.

In [34]:
df3[~df3['Type'].isin(["Paid", "Free"])]

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Command & Conquer: Rivals,FAMILY,NaN,0,NaN,0,NaN,0,Everyone 10+,Strategy,"June 28, 2018",NaN,NaN


In [35]:
print("No. of zero_price apps: {}\nNo. of free_type apps: {}\nNo. of non-zero_price apps: {}\nNo. of paid_type apps: {}"\
      .format(sum(df3["Price"] == "0"), 
              sum(df3["Type"] == "Free"), 
              sum(df3["Price"] != "0"),
              sum(df3["Type"] == "Paid")))
df3[(df3["Price"] == "0") & (df3["Type"] != "Free")]

No. of zero_price apps: 8906
No. of free_type apps: 8905
No. of non-zero_price apps: 754
No. of paid_type apps: 754


,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Command & Conquer: Rivals,FAMILY,NaN,0,NaN,0,NaN,0,Everyone 10+,Strategy,"June 28, 2018",NaN,NaN


In [36]:
df6 = df3.assign(Price_amt = lambda d: d.Price.str.extract('(\d+(\.\d+)?)')[0].astype(float))
df6.groupby('Type', group_keys = False).apply(lambda x: x.sample(min(len(x), 5))) # check results

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Price_amt
App,,,,,,,,,,,,,
Bacteria Vaginosis,HEALTH_AND_FITNESS,4.0,4015,8074035.2,1000000,Free,0,Mature 17+,Health & Fitness,"April 27, 2017",6.0,3.0,0.00
Lego Ninjago Wallpaper,MEDICAL,NaN,2,15728640.0,100,Free,0,Everyone,Medical,"August 2, 2018",1.2,3.0,0.00
Chess Free,GAME,4.5,1375988,15728640.0,50000000,Free,0,Everyone,Board,"June 7, 2018",2.72,4.1,0.00
Credit Sesame,FINANCE,4.3,25166,11534336.0,1000000,Free,0,Everyone,Finance,"July 25, 2018",3.1.16,4.1,0.00
video player for android,VIDEO_PLAYERS,4.5,351168,13631488.0,10000000,Free,0,Everyone,Video Players & Editors,"October 18, 2016",2.0.0,2.3,0.00
The EO Bar,HEALTH_AND_FITNESS,4.7,416,35651584.0,10000,Paid,$6.99,Everyone,Health & Fitness,"July 2, 2018",1.3.7,4.1,6.99
UFO-CQ,TOOLS,NaN,1,242688.0,10,Paid,$0.99,Everyone,Tools,"July 4, 2016",NaN,2.0,0.99
FN pistol model 1903 explained,BOOKS_AND_REFERENCE,NaN,1,19922944.0,10,Paid,$6.49,Everyone,Books & Reference,"September 5, 2015",3.0,1.6,6.49
Writing Wizard Premium - Handwriting,FAMILY,4.1,89,34603008.0,5000,Paid,$4.49,Everyone,Education;Education,"December 3, 2017",2.5.0,4.0.3,4.49


In [37]:
incomes = df6.Price_amt * df6.Installs
incomes.sort_values(ascending = False).head(20)

App
Minecraft                                           69900000.0
I am rich                                           39999000.0
I Am Rich Premium                                   19999500.0
Hitman Sniper                                        9900000.0
Grand Theft Auto: San Andreas                        6990000.0
Facetune - For Free                                  5990000.0
Sleep as Android Unlock                              5990000.0
DraStic DS Emulator                                  4990000.0
I'm Rich - Trump Edition                             4000000.0
I am Rich Plus                                       3999900.0
💎 I'm rich                                           3999900.0
I Am Rich                                            3899900.0
I am rich VIP                                        2999900.0
Weather Live                                         2995000.0
Card Wars - Adventure Time                           2990000.0
Threema                                            

#### (12) For each app, compute its minimum and maximum Sentiment_polarity

In [38]:
gps_ur.sample(10)

,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
App,,,,
FOX NOW - On Demand & Live TV,I get I get stuck stupid page I can't get of. ...,Positive,0.187273,0.700000
Alto's Adventure,"Guys, love game, graphics gameplay superb ads ...",Negative,-0.016667,0.657143
Gyft - Mobile Gift Card Wallet,This improved used update balances without req...,Positive,0.300000,0.550000
Diary with lock password,I think ok,Positive,0.500000,0.500000
Bank of America Mobile Banking,"I love secured security I get BOA app, also ea...",Positive,0.455556,0.755556
8 Ball Pool,NaN,NaN,NaN,NaN
Free & Premium VPN - FinchVPN,NaN,NaN,NaN,NaN
"Club Factory Everything, Unbeaten Price",NaN,NaN,NaN,NaN
Fast Secure VPN,81759,Neutral,0.000000,0.000000


In [39]:
len(gps_ur.index.unique())

1074

In [40]:
gps_ur.reset_index().groupby("App").agg({'Sentiment_Polarity': ['min', 'max']}).sort_values(("Sentiment_Polarity", "min"), ascending = False)

Sentiment_Polarity  \
                                                                  min   
App                                                                     
HomeWork                                                     1.000000   
Google Slides                                                0.933333   
Daily Workouts - Exercise Fitness Routine Trainer            0.800000   
Bed Time Fan - White Noise Sleep Sounds                      0.781250   
GPS Map Free                                                 0.700000   
Google Primer                                                0.700000   
Best Ovulation Tracker Fertility Calendar App Glow           0.595313   
All-in-One Mahjong 3 FREE                                    0.500000   
GPS Speedometer and Odometer                                 0.450000   
CallApp: Caller ID, Blocker & Phone Call Recorder            0.390741   
Brightest Flashlight Free ®                                  0.390000   
Drawing for Kids Learning Games for Toddlers age 3           0.386667   
Caller ID +                                                  0.350000   
FlipaClip - Cartoon animation                                0.337500   
Best Fiends - Free Puzzle Game                               0.325000   
Amazon Prime Video                                           0.322222   
Google+                                                      0.319444   
Calendar+ Schedule Planner App                               0.319071   
Google Trips - Travel Planner                                0.300000   
Apartment Decorating Ideas                                   0.245000   
Google Translate                                             0.188333   
Calculator - unit converter                                  0.183333   
CM Flashlight (Compass, SOS)                                 0.150000   
3D Live Neon Weed Launcher                                   0.136364   
Calculator Plus Free                                         0.133333   
Caf - Mon Compte                                             0.100000   
Couple - Relationship App                                    0.100000   
Draw a Stickman: EPIC 2                                      0.100000   
Daniel Tiger for Parents                                     0.072222   
7 Day Food Journal Challenge                                 0.070000   
...                                                               ...   
Girly Wallpapers Backgrounds                                      NaN   
Glam - Premium Dating App                                         NaN   
GlassesOff                                                        NaN   
Glide - Video Chat Messenger                                      NaN   
Glow Hockey                                                       NaN   
Glowing Flowers Live Wallpaper                                    NaN   
Google Allo                                                       NaN   
Google Analytics                                                  NaN   
Google Assistant                                                  NaN   
Google Assistant Go                                               NaN   
Google Calendar                                                   NaN   
Google Chrome: Fast & Secure                                      NaN   
Google Classroom                                                  NaN   
Google Docs                                                       NaN   
Google Drive                                                      NaN   
Google Play Books                                                 NaN   
Google Play Games                                                 NaN   
Google Play Movies & TV                                           NaN   
Google Sheets                                                     NaN   
HBO NOW: Stream TV & Movies                                       NaN   
HOTEL DEALS                                                       NaN   
HSL - Tickets, route planner and inf